In [544]:
import pandas as pd
import matplotlib.pyplot as plt

In [545]:
dtypes = {'Lote': str, 'Material': str, 'Lote interno': str, 'Lote Proveedor': str, 'Qty': float, 'Fecha recepción': object, 'Fecha traslado': object}

In [ ]:
mc = pd.read_excel('../../data/raw/Movimientos componentes.xlsx', dtype=dtypes)
mc.head()

In [ ]:
mc.info()

Fecha recepción y traslado are not datetimes

In [ ]:
mc.describe()

It is weird for a quantity to be less than 0 (even 0 is weird!)

In [ ]:
# Duplicated rows count
mc.duplicated().sum()

In [ ]:
# Lets take a look at the duplicated rows how many times they are duplicated
df = mc.copy()
# For each duplicated row we are going to count the number of times the row is duplicated
df['duplicated'] = df.duplicated(keep=False)
# Get those which are duplicated
df = df[df['duplicated']]
print(df[df['duplicated']].shape[0])
# Get the row number 1
#print(df.loc[df['Lote'] == df.iloc[1]['Lote']])
# Count the number of times the row is duplicated using apply, axis = 1

df['duplicated_count'] = df.apply(lambda x: df[df.eq(x).all(1)].shape[0], axis=1)
df = df.drop_duplicates()
df = df.sort_values(by='duplicated_count', ascending=False)
# Sum
df['duplicated_count'].sum()


In [ ]:
df.head(10)

In [ ]:
df.tail(10)

### Is there a pattern in the duplicated rows?

In [553]:
# Drop duplicated rows
mc.drop_duplicates(inplace=True)

# Columns


## Qty 

In [ ]:
# Is there any non-alphanumeric character in the 'Qty' column?
print('Number of values with digit characters: ' + str(mc['Qty'].astype(str).str.count('^[-0-9.]+').sum()))
# Which ones are positive?
print('Number of positive values: ' + str(mc[mc['Qty'] > 0]['Qty'].count()))
# Which ones are negative?
print('Number of negative values: ' + str(mc[mc['Qty'] < 0]['Qty'].count()))
# Which ones are zero?
print('Number of zero values: ' + str(mc[mc['Qty'] == 0]['Qty'].count()))

In [ ]:
# Lets show the negative values
mc[mc['Qty'] < 0]

In [ ]:
# Lets show the zero values
mc[mc['Qty'] == 0]

In [ ]:
# Lets show 10 of the positive values
mc[mc['Qty'] > 0].sample(10)

Weird values for recepción y traslado, lets check what else happened with those columns later.

## Lote

In [ ]:
### Are all lote chars digits?
mc['Lote'].str.isdigit().all()

In [ ]:
# Lote max and min length
print('Lote max length: ' + str(mc['Lote'].str.len().max()))
print('Lote min length: ' + str(mc['Lote'].str.len().min()))


In [ ]:
#Number of unique lote values
print('Number of unique lote values: ' + str(mc['Lote'].nunique()) + ' out of ' + str(mc['Lote'].count()))

194 unique values != 205 unique from OF 123456 excel sheet.

## Lote proveedor

In [ ]:
# Lote proveedor unique values
print('Number of unique lote proveedor values: ' + str(mc['Lote Proveedor'].nunique()) + 
      ' out of ' + str(mc['Lote Proveedor'].count()))

In [ ]:
# Min and max length 
print('Min lote length: ' + str(mc['Lote Proveedor'].str.len().min()))
print('Max lote length: ' + str(mc['Lote Proveedor'].str.len().max()))

In [ ]:
# Distribution of lengths
mc['Lote Proveedor'].str.len().value_counts().sort_index().plot(kind='bar')

In [ ]:
# They are not all digits, but they are all alphanumeric
mc['Lote Proveedor'].str.isalnum().all()

In [ ]:
# Which is not alphanumeric?
mc[~mc['Lote Proveedor'].str.isalnum()]['Lote Proveedor'].unique()

There are patterns and groups

In [ ]:
# First values of 'Lote Proveedor' patterns
mc['Lote Proveedor'].value_counts().index.str.extract(r'(^.{1})').value_counts()

In [ ]:
mc['Lote Proveedor'].value_counts().index.str.extract(r'(^.{2})').value_counts()

In [ ]:
mc['Lote Proveedor'].value_counts().index.str.extract(r'(^.{3})').value_counts()

In [ ]:
mc['Lote Proveedor'].value_counts().index.str.extract(r'(^.{4})').value_counts()

Could there be any value in splitting the suppliers into several categories for statistical analysis or ML related analysis?

## Lote interno

In [ ]:
# lote interno unique values
print('Number of unique lote interno values: ' + str(mc['Lote interno'].nunique()) + 
      ' out of ' + str(mc['Lote interno'].count()))

### This looks interesting, one would assume lote interno was an ID for the material transaction but seems it is not true

In [ ]:
# Are all lote interno chars digits?
mc['Lote interno'].str.isdigit().all()

In [ ]:
# Minimum and max length of lote interno
print('Minimum length of lote interno: ' + str(mc['Lote interno'].str.len().min()))
print('Maximum length of lote interno: ' + str(mc['Lote interno'].str.len().max()))

In [ ]:
# Is there any pattern to the unique values?
mc['Lote interno'].value_counts().index.str.extract(r'(^.{5})').value_counts()

In [ ]:
mc['Lote interno'].value_counts().index.str.extract(r'(.{6}$)').value_counts()

### This just looks like a numerical id, but we can't know what it is exactly identifying, it is likely a combination of Lote, Lote interno and Material/ proveedor which makes the row identifiable

## Fecha recepcion

In [ ]:
# Number of unique values for fecha recepción
print('Number of unique fecha recepción values: ' + str(mc['Fecha recepción'].nunique()) + 
      ' out of ' + str(mc['Fecha recepción'].count()))

In [ ]:
# Are all datetimes? 
# Check with type
import datetime
print('All recepcion datetimes: ' + str(mc['Fecha recepción'].apply(lambda x: type(x)).eq(datetime.datetime).all()))
print('All traslado datetimes: ' + str(mc['Fecha traslado'].apply(lambda x: type(x)).eq(datetime.datetime).all()))
# Get those which are not datetimes
not_dt = mc[~mc['Fecha recepción'].apply(lambda x: type(x)).eq(datetime.datetime)]
print('Number of not datetime values in fecha recepción: ' + str(not_dt.shape[0]))
not_dt = mc[~mc['Fecha traslado'].apply(lambda x: type(x)).eq(datetime.datetime)]
print('Number of not datetime values in fecha traslado: ' + str(not_dt.shape[0]))

not_dt.shape[0]
#nUnique of fecha recepción
print('nUnique of fecha recepción: ' + str(not_dt['Fecha recepción'].nunique()))
# nUnique of Lote
print('nUnique of Lote: ' + str(not_dt['Lote'].nunique()))  
# nUnique of Lote Proveedor
print('nUnique of Lote Proveedor: ' + str(not_dt['Lote Proveedor'].nunique()))
# nUnique of Material
print('nUnique of Material: ' + str(not_dt['Material'].nunique()))
# nUnique of Qty
print('nUnique of Qty: ' + str(not_dt['Qty'].nunique()))
# nUnique of Fecha traslado
print('nUnique of Fecha traslado: ' + str(not_dt['Fecha traslado'].nunique()))
# nUnique of Lote interno
print('nUnique of Lote interno: ' + str(not_dt['Lote interno'].nunique()))
print(not_dt)


In [ ]:
# Are non-datetime values in the fecha traslado greater than fecha recepción?
(not_dt['Fecha traslado'] > not_dt['Fecha recepción']).all()

## Duration

In [565]:
def dur_map(x):
    if type(x) == datetime.timedelta:
        # get days as int
        return x.days
    else:
        return x

In [ ]:
# Create a column for duration in mc
mc['Duration'] = mc['Fecha traslado'] - mc['Fecha recepción']
mc['Duration'] = mc['Duration'].apply(dur_map)
mc


In [ ]:
# Plot histogram of duration
mc[~mc.index.isin(not_dt.index)]['Duration'].hist(bins=100)
# Print mean
print('Mean duration: ' + str(mc[~mc.index.isin(not_dt.index)]['Duration'].mean()))
# Print median
print('Median duration: ' + str(mc[~mc.index.isin(not_dt.index)]['Duration'].median()))
# Print std
print('Std duration: ' + str(mc[~mc.index.isin(not_dt.index)]['Duration'].std()))

In [ ]:
# Plot histogram of duration
mc[mc.index.isin(not_dt.index)]['Duration'].hist(bins=100)
# Print mean
print('Mean duration: ' + str(mc[mc.index.isin(not_dt.index)]['Duration'].mean()))
# Print median
print('Median duration: ' + str(mc[mc.index.isin(not_dt.index)]['Duration'].median()))
# Print std
print('Std duration: ' + str(mc[mc.index.isin(not_dt.index)]['Duration'].std()))

In [ ]:
mc['Duration'].hist(bins=100)
# Print mean
print('Mean duration: ' + str(mc['Duration'].mean()))
# Print median
print('Median duration: ' + str(mc['Duration'].median()))
# Print std
print('Std duration: ' + str(mc['Duration'].std()))


## Fecha traslado


In [ ]:
# Unique values of Fecha traslado
print('Unique values of Fecha traslado: ' + str(mc['Fecha traslado'].nunique()) + ' out of '
                                                 + str(mc['Fecha traslado'].count()))

## Material

In [ ]:
print(mc['Material'].nunique())
mc['Material'].value_counts()

In [ ]:
# Plot histogram of Material
mc['Material'].value_counts().plot(kind='bar')

### Note: 100003 and 100013 as the least frequent

In [ ]:
## How many materials are there for each lote?
mc.groupby('Lote')['Material'].nunique().sort_values(ascending=False)

In [644]:
# Does the combination of material and lote uniquely identify a row?
mc.groupby(['Lote', 'Material']).size()



Lote   Material
23019  100001      3
       100002      3
       100003      1
       100004      2
       100005      2
                  ..
24110  100008      1
       100009      1
       100010      1
       100011      1
       100012      1
Length: 2327, dtype: int64

In [645]:
# Select those which Lote and material has more than one row
# Get lote 23019 and material 100001
mc[(mc['Lote'] == '23019') & (mc['Material'] == '100001')]

,Lote,Material,Lote interno,Lote Proveedor,Qty,Fecha recepción,Fecha traslado,Duration
0,23019,100001,0000737133,AM1798471,72.000,2023-01-12 00:00:00,2023-03-22 00:00:00,69
8,23019,100001,0000737133,AM1798471,57.600,2023-01-12 00:00:00,2023-03-02 00:00:00,49
19,23019,100001,0000737133,AM1798471,7.248,2023-01-12 00:00:00,2023-03-02 00:00:00,49


This example marks well what is happening here: We need all values to identify a row of this dataset.
We could make a new ID here to simplify it probably?